In [11]:
from copy import deepcopy
import os
import sys
import toml

In [12]:
from matplotlib.ticker import PercentFormatter
from tqdm.notebook import tqdm
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow.parquet as pa
import scipy.optimize as Opt
import torch

pd.options.display.float_format = '{:,.2f}'.format

In [30]:
sys.path.append(os.path.join(sys.path[0], '../..'))

from endure.data.io import Reader
from endure.lsm.cost import EndureCost
from endure.lcm.data.generator import ClassicGenerator
from endure.lsm.types import Policy, System, LSMDesign

In [32]:
config = Reader.read_config('../../endure.toml')

In [33]:
# fig, ax = plt.subplots(tight_layout=True)
# labels = ['Workload 1', 'Workload 2']
# width = 0.4
# cost_nom = [1, 5]
# cost_rob = [2, 3]
# xtick = np.arange(len(labels))
# xtick = [0, 0.5]
# rects1 = ax.bar(xtick - width / 2, cost_nom, width=width, label='Nominal', color='pink', hatch='.')
# rects1 = ax.bar(xtick, cost_nom, width=width, label='Nominal', color='pink', align='center')
# rects2 = ax.bar(xtick + width / 2, cost_rob, width=width, label='Robust', color='tab:orange', hatch='x')

# ax.set_yticks([])
# ax.set_ylabel('Cost', fontsize=20)
# ax.set_xticks(xtick, labels, fontsize=20)
# ax.legend(fontsize=24)

# fig.tight_layout()
# plt.show()

In [34]:
# fig, ax = plt.subplots(figsize=(100, 5), tight_layout=True)
# labels = ['Workload 1', 'Workload 2']
# width = 0.1
# cost_nom = np.random.rand(200)
# xtick = list(range(len(cost_nom)))
# cmap = plt.cm.tab10
# colors = cmap(np.arange(len(xtick)) % cmap.N)
# rects1 = ax.bar(xtick, cost_nom, align='center', color=colors)
# ax.set_yticks([])
# ax.set_xticks([])
# ax.set_ylabel('Cost', fontsize=20)
# fig.tight_layout()
# plt.show()

In [37]:
cost_fn = EndureCost(config)
gen = ClassicGenerator(config)

In [140]:
def obj(x, policy, system, z0, z1, q, w):
    h, T = x
    design = LSMDesign(h=h, T=T, policy=policy)
    cost = cost_fn.calc_cost(design, system, z0, z1, q, w)
    
    return cost

In [162]:
# z0, z1, q, w = gen._sample_workload(4)
z0, z1, q, w = (0.024, 0.291, 0.214, 0.471)
system = System()
policy = Policy.Leveling

minimizer_kwargs = {
    'method': 'SLSQP',
    'bounds': Opt.Bounds(lb=np.array((0, 2)), ub=np.array((9, 50)), keep_feasible=(True, True)),
    # 'bounds': ((0, 9), (2, 50)),
    'options': {'disp': True}
}

sol = Opt.minimize(fun=lambda x: obj(x, policy, system, z0, z1, q, w), x0=[2, -10000], callback=lambda x: print(f'{x=}'), **minimizer_kwargs)
sol

x=array([2.0078963 , 4.24207711])
x=array([1.97571143, 4.25265447])
x=array([1.81744949, 4.29960066])
x=array([1.11802606, 4.48643268])
x=array([0.75935298, 4.56109556])
x=array([0.74329571, 4.54678747])
x=array([0.71632988, 4.47086769])
x=array([0.69801654, 4.42437615])
x=array([0.69801654, 4.42437615])
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.103249097367497
            Iterations: 9
            Function evaluations: 27
            Gradient evaluations: 9


     fun: 9.103249097367497
     jac: array([-3.49283218e-05, -3.12566757e-04])
 message: 'Optimization terminated successfully'
    nfev: 27
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([0.69801654, 4.42437615])

In [168]:
a = torch.Tensor([1, 2, 3])

In [169]:
a = a.clamp(0, 1)
a

tensor([1., 1., 1.])

In [175]:
a = {'a': 2, 'b': 3}
a

{'a': 2, 'b': 3}

In [34]:
import torch
import torch.nn.functional as F


def round_func_normal(input):
    out = torch.round(input)
    return out


def round_func_BPDA(input):
    # This is equivalent to replacing round function (non-differentiable) with
    # an identity function (differentiable) only when backward.
    forward_value = torch.round(input)
    out = input.clone()
    out.data = forward_value.data
    return out


def forward(x, round_func):
    linear = torch.nn.Linear(2, 1)
    linear.weight.data = torch.Tensor([-0.3917, 0.2402]).view(1, -1)
    linear.bias.data = torch.Tensor([-0.3856]).view(1, -1)
    out = linear(x)            # differentiable
    out = out * 10             # differentiable
    out = round_func(out)      # defended by non-differentiable operation (shattered gradients)
    out = out * 0.01           # differentiable
    out = torch.sigmoid(out)   # differentiable
    return out

In [35]:
# scenario 1: No Defence
x = torch.Tensor([4, -1.12]).view(1, -1).requires_grad_(True)
out = forward(x, lambda x: x)
loss = F.binary_cross_entropy(out, torch.tensor([1.]).view(1, -1))
loss.backward()
print(loss)           # tensor(0.8104, grad_fn=<BinaryCrossEntropyBackward>)
print(x.grad)         # tensor([[[ 0.0218, -0.0133]]])

tensor(0.8104, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([[ 0.0218, -0.0133]])


In [36]:
# scenario 2: Defended by round function (shattered gradients)
x = torch.tensor([4, -1.12]).view(1, -1).requires_grad_(True)
out = forward(x, round_func_normal)
loss = F.binary_cross_entropy(out, torch.tensor([1.]).view(1, -1))
loss.backward()
print(loss)           # tensor(0.8092, grad_fn=<BinaryCrossEntropyBackward>)
print(x.grad)         # tensor([[[-0., 0.]]])

tensor(0.8092, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([[0., 0.]])


In [37]:
# scenario 3: Defended by round function, attached by BPDA
x = torch.tensor([4, -1.12]).view(1, -1).requires_grad_(True)
out = forward(x, round_func_BPDA)
loss = F.binary_cross_entropy(out, torch.tensor([1.]).view(1, -1))
loss.backward()
print(loss)           # tensor(0.8092, grad_fn=<BinaryCrossEntropyBackward>)
print(x.grad)         # tensor([[[ 0.0217, -0.0133]]])

tensor(0.8092, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([[ 0.0217, -0.0133]])
